# Day 05: Streaming Data Processing for Trading Systems

## Week 22: System Design

This notebook covers streaming data processing techniques essential for building real-time trading systems. We'll explore:

- **Stream generators** for simulating market data feeds
- **Sliding window processors** for efficient data buffering
- **Real-time calculations** (moving averages, VWAP, etc.)
- **Anomaly detection** on streaming data
- **Async processing** for concurrent data handling

### Why Streaming Matters in Finance

Trading systems require real-time data processing with:
- **Low latency**: Milliseconds matter for price updates
- **High throughput**: Thousands of ticks per second
- **Memory efficiency**: Can't store all historical data in RAM
- **Continuous computation**: Rolling statistics, signals, risk metrics

## 1. Import Required Libraries

In [ ]:
"""
Streaming Data Processing Libraries
====================================
Core libraries for building streaming data pipelines in Python.
"""

import asyncio
import random
import time
from collections import deque
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from typing import (
    AsyncGenerator,
    Callable,
    Deque,
    Dict,
    Generator,
    List,
    Optional,
    Tuple,
    TypeVar,
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For Jupyter async support
import nest_asyncio
nest_asyncio.apply()

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Data Structures for Streaming

### Tick Data Structure

First, let's define a standard data structure for market data ticks:

In [ ]:
@dataclass
class Tick:
    """
    Represents a single market data tick.
    
    Attributes:
        timestamp: When the tick occurred
        symbol: Trading instrument identifier
        price: Trade/quote price
        volume: Trade size or quote quantity
        side: 'bid', 'ask', or 'trade'
    """
    timestamp: datetime
    symbol: str
    price: float
    volume: float
    side: str = 'trade'
    
    def __repr__(self):
        return f"Tick({self.symbol} @ {self.price:.2f} x {self.volume:.0f})"


@dataclass 
class OHLCV:
    """
    OHLCV bar for aggregated price data.
    """
    timestamp: datetime
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: float
    trade_count: int = 0
    
    def __repr__(self):
        return f"OHLCV({self.symbol} O:{self.open:.2f} H:{self.high:.2f} L:{self.low:.2f} C:{self.close:.2f} V:{self.volume:.0f})"


# Example tick
tick = Tick(
    timestamp=datetime.now(),
    symbol="AAPL",
    price=185.50,
    volume=100,
    side='trade'
)
print(f"Example tick: {tick}")

## 3. Simple Data Stream Generator

A generator function that yields simulated price ticks. Generators are memory-efficient as they produce data on-demand rather than storing everything in memory.

In [ ]:
def price_tick_generator(
    symbol: str = "AAPL",
    initial_price: float = 100.0,
    volatility: float = 0.02,
    num_ticks: int = 1000,
    ticks_per_second: float = 10.0
) -> Generator[Tick, None, None]:
    """
    Generate simulated price ticks using geometric Brownian motion.
    
    Parameters:
        symbol: Trading symbol
        initial_price: Starting price
        volatility: Price volatility (std dev of returns)
        num_ticks: Number of ticks to generate
        ticks_per_second: Simulated tick frequency
        
    Yields:
        Tick objects with timestamp, price, and volume
    """
    price = initial_price
    base_time = datetime.now()
    
    for i in range(num_ticks):
        # Simulate price movement (GBM-like)
        returns = np.random.normal(0, volatility)
        price = price * (1 + returns)
        
        # Simulate volume (log-normal distribution)
        volume = int(np.exp(np.random.normal(5, 1)))  # ~150 avg volume
        
        # Create timestamp
        timestamp = base_time + timedelta(seconds=i / ticks_per_second)
        
        yield Tick(
            timestamp=timestamp,
            symbol=symbol,
            price=price,
            volume=volume,
            side=random.choice(['trade', 'bid', 'ask'])
        )


# Test the generator
print("First 10 ticks from generator:")
print("-" * 60)

gen = price_tick_generator(num_ticks=10)
for tick in gen:
    print(f"{tick.timestamp.strftime('%H:%M:%S.%f')[:-3]} | {tick}")

## 4. Sliding Window Processor

A sliding window maintains a fixed-size buffer of the most recent data points. Using `deque` with `maxlen` provides O(1) append and automatic eviction of old data.

In [ ]:
T = TypeVar('T')

class SlidingWindow:
    """
    Fixed-size sliding window using deque for O(1) operations.
    
    Features:
    - Automatic eviction when window is full
    - Memory-efficient for streaming data
    - Supports iteration and indexing
    """
    
    def __init__(self, window_size: int):
        """
        Initialize sliding window.
        
        Parameters:
            window_size: Maximum number of elements in window
        """
        self.window_size = window_size
        self._buffer: Deque = deque(maxlen=window_size)
        
    def add(self, item) -> Optional[object]:
        """
        Add item to window. Returns evicted item if window was full.
        """
        evicted = None
        if len(self._buffer) == self.window_size:
            evicted = self._buffer[0]  # Will be evicted
        self._buffer.append(item)
        return evicted
    
    def is_full(self) -> bool:
        """Check if window has reached capacity."""
        return len(self._buffer) == self.window_size
    
    def get_data(self) -> List:
        """Return copy of window data as list."""
        return list(self._buffer)
    
    def __len__(self) -> int:
        return len(self._buffer)
    
    def __getitem__(self, index):
        return self._buffer[index]
    
    def __iter__(self):
        return iter(self._buffer)


# Demonstrate sliding window
print("Sliding Window Demo (size=5):")
print("-" * 40)

window = SlidingWindow(window_size=5)

for i in range(8):
    evicted = window.add(i * 10)
    print(f"Added: {i*10:3d} | Window: {window.get_data()} | Evicted: {evicted}")

## 5. Real-Time Moving Average Calculator

An efficient incremental SMA that updates with each new data point without recalculating the entire window. This is crucial for low-latency systems.

In [ ]:
class StreamingSMA:
    """
    Streaming Simple Moving Average with O(1) update complexity.
    
    Instead of recalculating sum each time, we:
    1. Subtract the oldest value (being evicted)
    2. Add the new value
    3. Divide by window size
    """
    
    def __init__(self, window_size: int):
        self.window_size = window_size
        self._window: Deque[float] = deque(maxlen=window_size)
        self._sum: float = 0.0
        
    def update(self, value: float) -> Optional[float]:
        """
        Add new value and return updated SMA.
        
        Returns:
            Current SMA, or None if window not yet full
        """
        # Subtract evicted value if window is full
        if len(self._window) == self.window_size:
            self._sum -= self._window[0]
            
        # Add new value
        self._window.append(value)
        self._sum += value
        
        # Return SMA only when window is full
        if len(self._window) == self.window_size:
            return self._sum / self.window_size
        return None
    
    @property
    def current_sma(self) -> Optional[float]:
        """Get current SMA value."""
        if len(self._window) == self.window_size:
            return self._sum / self.window_size
        return None
    
    def __repr__(self):
        return f"StreamingSMA(window={self.window_size}, sma={self.current_sma})"


# Demo: Streaming SMA on price data
print("Streaming SMA Demo (window=5):")
print("-" * 60)

sma = StreamingSMA(window_size=5)
prices = [100, 102, 101, 103, 105, 104, 106, 108, 107, 109]

for i, price in enumerate(prices):
    result = sma.update(price)
    status = f"{result:.2f}" if result else "warming up..."
    print(f"Tick {i+1:2d}: Price={price:6.2f} | SMA={status}")

## 6. Exponential Moving Average Stream Processor

EMA applies exponential weighting to streaming data. It's stateless (no window storage needed) and responds faster to recent price changes.

**Formula**: $EMA_t = \alpha \cdot price_t + (1 - \alpha) \cdot EMA_{t-1}$

Where $\alpha = \frac{2}{span + 1}$

In [ ]:
class StreamingEMA:
    """
    Streaming Exponential Moving Average.
    
    Advantages over SMA:
    - No window storage needed (memory efficient)
    - Responds faster to recent changes
    - O(1) time and O(1) space complexity
    
    Parameters:
        span: EMA span (similar to SMA window)
        alpha: Smoothing factor (calculated from span if not provided)
    """
    
    def __init__(self, span: int = None, alpha: float = None):
        if alpha is not None:
            self.alpha = alpha
        elif span is not None:
            self.alpha = 2.0 / (span + 1)
        else:
            raise ValueError("Either span or alpha must be provided")
            
        self.span = span
        self._ema: Optional[float] = None
        self._count: int = 0
        
    def update(self, value: float) -> float:
        """
        Update EMA with new value.
        
        Returns:
            Updated EMA value
        """
        self._count += 1
        
        if self._ema is None:
            # First value: EMA = value
            self._ema = value
        else:
            # EMA update formula
            self._ema = self.alpha * value + (1 - self.alpha) * self._ema
            
        return self._ema
    
    @property
    def current_ema(self) -> Optional[float]:
        return self._ema
    
    def __repr__(self):
        return f"StreamingEMA(alpha={self.alpha:.4f}, ema={self._ema})"


# Demo: Compare SMA and EMA
print("SMA vs EMA Comparison (span=5):")
print("-" * 70)

sma = StreamingSMA(window_size=5)
ema = StreamingEMA(span=5)

prices = [100, 102, 101, 103, 105, 104, 106, 108, 107, 109]

print(f"{'Tick':>4} | {'Price':>8} | {'SMA':>10} | {'EMA':>10}")
print("-" * 48)

for i, price in enumerate(prices):
    sma_val = sma.update(price)
    ema_val = ema.update(price)
    
    sma_str = f"{sma_val:.2f}" if sma_val else "---"
    print(f"{i+1:4d} | {price:8.2f} | {sma_str:>10} | {ema_val:10.2f}")

## 7. Stream Buffer with Batching

For improved throughput, we can batch incoming data points and process them in chunks. This reduces per-tick overhead and enables vectorized operations.

In [ ]:
class BatchedStreamProcessor:
    """
    Buffers incoming data and processes in batches.
    
    Benefits:
    - Reduced function call overhead
    - Enables vectorized (NumPy) operations
    - Configurable batch size for latency vs throughput tradeoff
    """
    
    def __init__(
        self, 
        batch_size: int, 
        processor: Callable[[List], any],
        flush_on_incomplete: bool = True
    ):
        """
        Parameters:
            batch_size: Number of items per batch
            processor: Function to process each batch
            flush_on_incomplete: Whether to process incomplete final batch
        """
        self.batch_size = batch_size
        self.processor = processor
        self.flush_on_incomplete = flush_on_incomplete
        self._buffer: List = []
        self._results: List = []
        
    def add(self, item) -> Optional[any]:
        """
        Add item to buffer. Process batch when full.
        
        Returns:
            Batch result if batch was processed, None otherwise
        """
        self._buffer.append(item)
        
        if len(self._buffer) >= self.batch_size:
            return self._process_batch()
        return None
    
    def _process_batch(self) -> any:
        """Process current buffer and clear it."""
        batch = self._buffer[:self.batch_size]
        self._buffer = self._buffer[self.batch_size:]
        
        result = self.processor(batch)
        self._results.append(result)
        return result
    
    def flush(self) -> Optional[any]:
        """Process any remaining items in buffer."""
        if self._buffer and self.flush_on_incomplete:
            result = self.processor(self._buffer)
            self._results.append(result)
            self._buffer = []
            return result
        return None
    
    @property
    def results(self) -> List:
        return self._results


# Demo: Batch processing with statistics
def compute_batch_stats(batch: List[float]) -> Dict:
    """Compute statistics for a batch of prices."""
    arr = np.array(batch)
    return {
        'count': len(batch),
        'mean': np.mean(arr),
        'std': np.std(arr),
        'min': np.min(arr),
        'max': np.max(arr)
    }

print("Batched Stream Processing Demo (batch_size=5):")
print("-" * 60)

processor = BatchedStreamProcessor(
    batch_size=5,
    processor=compute_batch_stats
)

# Simulate streaming data
prices = [100, 102, 101, 103, 105, 104, 106, 108, 107, 109, 110, 112]

for i, price in enumerate(prices):
    result = processor.add(price)
    if result:
        print(f"Batch processed at tick {i+1}: {result}")

# Flush remaining
final = processor.flush()
if final:
    print(f"Final batch flushed: {final}")

## 8. VWAP Stream Processor

Volume Weighted Average Price (VWAP) is a key benchmark in trading. We implement an incremental VWAP calculator that processes streaming price and volume data.

**Formula**: $VWAP = \frac{\sum_{i=1}^{n} Price_i \times Volume_i}{\sum_{i=1}^{n} Volume_i}$

In [ ]:
class StreamingVWAP:
    """
    Streaming Volume Weighted Average Price calculator.
    
    Supports:
    - Cumulative VWAP (since market open)
    - Rolling VWAP (fixed window)
    - Anchored VWAP (from specific point)
    """
    
    def __init__(self, window_size: Optional[int] = None):
        """
        Parameters:
            window_size: If None, compute cumulative VWAP.
                        Otherwise, rolling VWAP over window.
        """
        self.window_size = window_size
        
        if window_size:
            # Rolling VWAP with sliding window
            self._prices: Deque[float] = deque(maxlen=window_size)
            self._volumes: Deque[float] = deque(maxlen=window_size)
            self._pv_sum: float = 0.0  # Price * Volume sum
            self._v_sum: float = 0.0   # Volume sum
        else:
            # Cumulative VWAP
            self._pv_sum: float = 0.0
            self._v_sum: float = 0.0
            
        self._tick_count: int = 0
        
    def update(self, price: float, volume: float) -> float:
        """
        Add new tick and return updated VWAP.
        
        Parameters:
            price: Trade price
            volume: Trade volume
            
        Returns:
            Current VWAP
        """
        self._tick_count += 1
        pv = price * volume
        
        if self.window_size:
            # Rolling VWAP: adjust for evicted data
            if len(self._prices) == self.window_size:
                old_pv = self._prices[0] * self._volumes[0]
                old_v = self._volumes[0]
                self._pv_sum -= old_pv
                self._v_sum -= old_v
                
            self._prices.append(price)
            self._volumes.append(volume)
            self._pv_sum += pv
            self._v_sum += volume
        else:
            # Cumulative VWAP
            self._pv_sum += pv
            self._v_sum += volume
            
        if self._v_sum > 0:
            return self._pv_sum / self._v_sum
        return price
    
    @property
    def current_vwap(self) -> float:
        if self._v_sum > 0:
            return self._pv_sum / self._v_sum
        return 0.0
    
    @property
    def total_volume(self) -> float:
        return self._v_sum
    
    def reset(self):
        """Reset VWAP (e.g., for new trading day)."""
        if self.window_size:
            self._prices.clear()
            self._volumes.clear()
        self._pv_sum = 0.0
        self._v_sum = 0.0
        self._tick_count = 0


# Demo: Streaming VWAP
print("Streaming VWAP Demo:")
print("-" * 70)

# Create cumulative and rolling VWAP calculators
vwap_cumulative = StreamingVWAP(window_size=None)
vwap_rolling = StreamingVWAP(window_size=5)

# Simulated trade data
trades = [
    (100.00, 1000), (100.50, 500), (99.75, 2000), (100.25, 800),
    (100.75, 1500), (101.00, 600), (100.50, 1200), (101.25, 900),
    (100.80, 1100), (101.50, 700)
]

print(f"{'Trade':>5} | {'Price':>8} | {'Volume':>8} | {'VWAP(cum)':>10} | {'VWAP(5)':>10}")
print("-" * 60)

for i, (price, volume) in enumerate(trades):
    vc = vwap_cumulative.update(price, volume)
    vr = vwap_rolling.update(price, volume)
    print(f"{i+1:5d} | {price:8.2f} | {volume:8d} | {vc:10.4f} | {vr:10.4f}")

## 9. Streaming Anomaly Detection

Detect outliers in real-time using rolling z-score. Useful for identifying unusual price movements, potential data errors, or trading opportunities.

In [ ]:
class StreamingAnomalyDetector:
    """
    Real-time anomaly detection using rolling statistics.
    
    Methods:
    - Z-score: Flag values > k standard deviations from mean
    - IQR: Flag values outside Q1 - 1.5*IQR to Q3 + 1.5*IQR
    - Adaptive threshold: Dynamic threshold based on recent volatility
    """
    
    def __init__(
        self, 
        window_size: int = 20,
        z_threshold: float = 3.0,
        method: str = 'zscore'
    ):
        """
        Parameters:
            window_size: Lookback window for statistics
            z_threshold: Number of std devs for anomaly
            method: 'zscore' or 'iqr'
        """
        self.window_size = window_size
        self.z_threshold = z_threshold
        self.method = method
        
        self._window: Deque[float] = deque(maxlen=window_size)
        
        # Welford's algorithm for online variance
        self._count: int = 0
        self._mean: float = 0.0
        self._M2: float = 0.0  # Sum of squared differences
        
        self._anomalies: List[Tuple[int, float, float]] = []
        
    def update(self, value: float) -> Tuple[bool, Optional[float]]:
        """
        Check if value is anomaly and update statistics.
        
        Returns:
            (is_anomaly, z_score)
        """
        is_anomaly = False
        z_score = None
        
        if len(self._window) >= self.window_size:
            # Calculate z-score against current window
            mean = np.mean(self._window)
            std = np.std(self._window)
            
            if std > 0:
                z_score = (value - mean) / std
                
                if self.method == 'zscore':
                    is_anomaly = abs(z_score) > self.z_threshold
                elif self.method == 'iqr':
                    q1, q3 = np.percentile(list(self._window), [25, 75])
                    iqr = q3 - q1
                    lower = q1 - 1.5 * iqr
                    upper = q3 + 1.5 * iqr
                    is_anomaly = value < lower or value > upper
                    
        # Update window
        self._window.append(value)
        self._count += 1
        
        if is_anomaly:
            self._anomalies.append((self._count, value, z_score))
            
        return is_anomaly, z_score
    
    @property
    def anomaly_count(self) -> int:
        return len(self._anomalies)
    
    @property
    def anomalies(self) -> List[Tuple[int, float, float]]:
        return self._anomalies


# Demo: Anomaly detection on streaming data
print("Streaming Anomaly Detection Demo:")
print("-" * 70)

detector = StreamingAnomalyDetector(window_size=10, z_threshold=2.5)

# Generate data with some anomalies
np.random.seed(42)
normal_prices = 100 + np.cumsum(np.random.normal(0, 0.5, 50))

# Inject anomalies
anomaly_indices = [15, 25, 40]
for idx in anomaly_indices:
    normal_prices[idx] += np.random.choice([-1, 1]) * 5  # Big spike

print(f"{'Tick':>4} | {'Price':>8} | {'Z-Score':>10} | {'Anomaly':>8}")
print("-" * 42)

for i, price in enumerate(normal_prices):
    is_anomaly, z_score = detector.update(price)
    z_str = f"{z_score:.2f}" if z_score else "---"
    anomaly_flag = "⚠️ YES" if is_anomaly else ""
    
    if is_anomaly or i < 5 or i in anomaly_indices:
        print(f"{i+1:4d} | {price:8.2f} | {z_str:>10} | {anomaly_flag}")

print(f"\n📊 Total anomalies detected: {detector.anomaly_count}")

## 10. OHLCV Bar Aggregator

Aggregate streaming tick data into OHLCV bars. Supports time-based (1min, 5min) and volume-based (1000 shares) bars.

In [ ]:
class OHLCVAggregator:
    """
    Aggregate streaming ticks into OHLCV bars.
    
    Supports multiple bar types:
    - Time bars: Fixed time intervals (1min, 5min, etc.)
    - Volume bars: Fixed volume threshold
    - Tick bars: Fixed number of ticks
    - Dollar bars: Fixed dollar volume
    """
    
    def __init__(
        self,
        symbol: str,
        bar_type: str = 'time',  # 'time', 'volume', 'tick', 'dollar'
        bar_size: int = 60,      # seconds for time, shares for volume, etc.
    ):
        self.symbol = symbol
        self.bar_type = bar_type
        self.bar_size = bar_size
        
        self._bars: List[OHLCV] = []
        self._reset_current_bar()
        
    def _reset_current_bar(self, timestamp: datetime = None):
        """Reset current bar accumulator."""
        self._bar_start = timestamp
        self._open: Optional[float] = None
        self._high: float = float('-inf')
        self._low: float = float('inf')
        self._close: float = 0.0
        self._volume: float = 0.0
        self._dollar_volume: float = 0.0
        self._tick_count: int = 0
        
    def _should_close_bar(self, tick: Tick) -> bool:
        """Check if current bar should be closed."""
        if self._open is None:
            return False
            
        if self.bar_type == 'time':
            elapsed = (tick.timestamp - self._bar_start).total_seconds()
            return elapsed >= self.bar_size
        elif self.bar_type == 'volume':
            return self._volume >= self.bar_size
        elif self.bar_type == 'tick':
            return self._tick_count >= self.bar_size
        elif self.bar_type == 'dollar':
            return self._dollar_volume >= self.bar_size
        return False
    
    def _create_bar(self) -> OHLCV:
        """Create OHLCV bar from current accumulator."""
        return OHLCV(
            timestamp=self._bar_start,
            symbol=self.symbol,
            open=self._open,
            high=self._high,
            low=self._low,
            close=self._close,
            volume=self._volume,
            trade_count=self._tick_count
        )
        
    def update(self, tick: Tick) -> Optional[OHLCV]:
        """
        Process new tick. Returns completed bar if bar closed.
        """
        completed_bar = None
        
        # Check if we should close current bar
        if self._should_close_bar(tick):
            completed_bar = self._create_bar()
            self._bars.append(completed_bar)
            self._reset_current_bar(tick.timestamp)
            
        # Update current bar
        if self._open is None:
            self._bar_start = tick.timestamp
            self._open = tick.price
            
        self._high = max(self._high, tick.price)
        self._low = min(self._low, tick.price)
        self._close = tick.price
        self._volume += tick.volume
        self._dollar_volume += tick.price * tick.volume
        self._tick_count += 1
        
        return completed_bar
    
    def flush(self) -> Optional[OHLCV]:
        """Force close current bar (e.g., end of session)."""
        if self._open is not None:
            bar = self._create_bar()
            self._bars.append(bar)
            self._reset_current_bar()
            return bar
        return None
    
    @property
    def bars(self) -> List[OHLCV]:
        return self._bars


# Demo: Tick to OHLCV aggregation
print("Tick to OHLCV Bar Aggregation Demo:")
print("-" * 70)

# Create aggregators for different bar types
tick_agg = OHLCVAggregator("AAPL", bar_type='tick', bar_size=10)
volume_agg = OHLCVAggregator("AAPL", bar_type='volume', bar_size=500)

# Generate ticks
gen = price_tick_generator(symbol="AAPL", num_ticks=50)

print("\nTick Bars (10 ticks each):")
print("-" * 60)
for tick in gen:
    bar = tick_agg.update(tick)
    if bar:
        print(bar)

# Flush remaining
final_bar = tick_agg.flush()
if final_bar:
    print(f"Final bar: {final_bar}")
    
print(f"\n📊 Total bars created: {len(tick_agg.bars)}")

## 11. Async Stream Consumer

Asynchronous stream processing using `asyncio` for handling multiple concurrent data streams. Essential for real-world trading systems that consume from multiple sources.

In [ ]:
async def async_price_stream(
    symbol: str,
    initial_price: float = 100.0,
    num_ticks: int = 10,
    delay_ms: float = 100
) -> AsyncGenerator[Tick, None]:
    """
    Async generator for simulated price stream.
    
    Parameters:
        symbol: Trading symbol
        initial_price: Starting price
        num_ticks: Number of ticks to generate
        delay_ms: Milliseconds between ticks
    """
    price = initial_price
    
    for i in range(num_ticks):
        # Simulate network/exchange delay
        await asyncio.sleep(delay_ms / 1000)
        
        # Price movement
        price *= (1 + np.random.normal(0, 0.01))
        volume = int(np.exp(np.random.normal(5, 1)))
        
        yield Tick(
            timestamp=datetime.now(),
            symbol=symbol,
            price=price,
            volume=volume
        )


async def process_single_stream(symbol: str, num_ticks: int = 5):
    """Process a single async price stream."""
    print(f"\n📡 Starting stream for {symbol}...")
    
    async for tick in async_price_stream(symbol, num_ticks=num_ticks):
        print(f"  [{symbol}] {tick.timestamp.strftime('%H:%M:%S.%f')[:-3]} | "
              f"Price: {tick.price:.2f} | Vol: {tick.volume}")
        
    print(f"  [{symbol}] Stream ended")


async def process_multiple_streams():
    """
    Process multiple price streams concurrently.
    Demonstrates parallel data consumption.
    """
    print("🚀 Processing multiple streams concurrently...")
    
    # Create tasks for multiple symbols
    tasks = [
        process_single_stream("AAPL", num_ticks=3),
        process_single_stream("GOOGL", num_ticks=3),
        process_single_stream("MSFT", num_ticks=3),
    ]
    
    # Run all streams concurrently
    await asyncio.gather(*tasks)
    
    print("\n✅ All streams completed!")


# Run the async demo
print("Async Stream Processing Demo:")
print("-" * 60)
asyncio.run(process_multiple_streams())